In [2]:
#This file will examine charts of the top 200 songs globally and from 62 countries for the week of 07/10/2020
#to 07/16/20 to determine if there is a significant relation between where a song becomes popular and its global
#relevance.


#bringing in dependencies
import pandas as pd
import glob
import os
import csv
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np
import sys
import json
import re

In [3]:
#bringing in global chart
globalchart = os.path.join("WeeklyCharts", "Global Weekly Chart.csv")

#making it readable with pandas library
globalchart_df = pd.read_csv(globalchart)
globalchart_df.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.,Unnamed: 4
0,Position,Track Name,Artist,Streams,URL
1,1,ROCKSTAR (feat. Roddy Ricch),DaBaby,36999793,https://open.spotify.com/track/7ytR5pFWmSjzHJI...
2,2,Savage Love (Laxed - Siren Beat),Jawsh 685,34262951,https://open.spotify.com/track/1xQ6trAsedVPCdb...
3,3,Blinding Lights,The Weeknd,30727192,https://open.spotify.com/track/0VjIjW4GlUZAMYd...
4,4,Come & Go (with Marshmello),Juice WRLD,30027587,https://open.spotify.com/track/7y7w4tl4MaRC2UM...


In [4]:
#cleaning up chart a bit by removing unnecessary columns and correcting headers
globalchart_clean_df = globalchart_df.drop(columns=['Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.', 'Unnamed: 4'])
globalchart_clean_df.rename(columns = {'Unnamed: 0' : 'Global - Rank', 'Unnamed: 1' : 'Name of Track', 'Unnamed: 2' : 'Artist'}, inplace=True)


globalchart_clean_df


,Global - Rank,Name of Track,Artist
0,Position,Track Name,Artist
1,1,ROCKSTAR (feat. Roddy Ricch),DaBaby
2,2,Savage Love (Laxed - Siren Beat),Jawsh 685
3,3,Blinding Lights,The Weeknd
4,4,Come & Go (with Marshmello),Juice WRLD
...,...,...,...
196,196,El Mismo Aire - con Pablo Alborán,Camilo
197,197,Lalala,Y2K
198,198,Sigues Con El,Dímelo Flow
199,199,Something Special,Pop Smoke


In [5]:
#Now working on making the data more readable in the files so that we can look at it holistically

#accessing all files in folder
allFiles = glob.glob(os.path.join("WeeklyCharts","*.csv"))

#adding them to a list
chart_list = []

for file in allFiles:
    chart_list.append(file)
chart_list.sort()


#getting a list of just the countries for later
country_list = []

for country in chart_list:
    name = country
    y = re.split('/| Weekly C|', name)
    country_list.append(y[1])
country_list.sort()


/Applications/anaconda3/envs/PythonData/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [35]:
#Now let's repeat the process we did with the global chart with all the countries' charts

#establishing counter for country header labels
i = 0

#setting up final df before loop
total_chart_df = pd.DataFrame()
total_chart_df = pd.concat([globalchart_clean_df, total_chart_df], axis = 1)

#looping to save time yay
for country in chart_list:
    
    country_df = pd.read_csv(country)
    
#still struggling to have same command on different lines so it's just long, sorry
    country_df = country_df.drop(columns=['Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.', 'Unnamed: 4'])

#Is there a cleaner way to do this? Maybe having headers for each DF, but wasn't sure how to go
#about that so formatting is a little iffy
    country_df = country_df.rename(columns = {'Unnamed: 0' : country_list[i] + ' - Rank', 'Unnamed: 1' : 'Name of Track', 'Unnamed: 2' : 'Artist'})
 
    i = i + 1
    #total_chart_df = pd.concat([total_chart_df, country_df], axis=1) #this is for a full data set
    total_chart_df = pd.merge(total_chart_df, country_df, how='left', on="Name of Track") #this merges on song name
#removing duplicate labelings
total_chart_df = total_chart_df.iloc[1:]

#final chart
total_chart_df

#total_chart_df_t = total_chart_df.transpose()
#total_chart_df_t



,Global - Rank_x,Name of Track,Artist_x,Argentina - Rank,Artist_y,Australia - Rank,Artist_x,Austria - Rank,Artist_y,Belgium - Rank,...,Turkey - Rank,Artist_y,UK - Rank,Artist_x,USA - Rank,Artist_y,Uruguay - Rank,Artist_x,Vietnam - Rank,Artist_y
1,1,ROCKSTAR (feat. Roddy Ricch),DaBaby,62,DaBaby,1,DaBaby,3,DaBaby,1,...,139,DaBaby,1,DaBaby,7,DaBaby,75,DaBaby,129,DaBaby
2,2,Savage Love (Laxed - Siren Beat),Jawsh 685,129,Jawsh 685,2,Jawsh 685,1,Jawsh 685,2,...,NaN,NaN,2,Jawsh 685,29,Jawsh 685,137,Jawsh 685,28,Jawsh 685
3,3,Blinding Lights,The Weeknd,23,The Weeknd,7,The Weeknd,10,The Weeknd,4,...,37,The Weeknd,8,The Weeknd,23,The Weeknd,33,The Weeknd,92,The Weeknd
4,4,Come & Go (with Marshmello),Juice WRLD,NaN,NaN,3,Juice WRLD,11,Juice WRLD,26,...,NaN,NaN,3,Juice WRLD,1,Juice WRLD,NaN,NaN,82,Juice WRLD
5,5,Watermelon Sugar,Harry Styles,22,Harry Styles,5,Harry Styles,12,Harry Styles,7,...,NaN,NaN,4,Harry Styles,22,Harry Styles,38,Harry Styles,147,Harry Styles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98756,196,El Mismo Aire - con Pablo Alborán,Camilo,35,Camilo,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,21,Camilo,NaN,NaN
98757,197,Lalala,Y2K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98758,198,Sigues Con El,Dímelo Flow,44,Dímelo Flow,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,56,Dímelo Flow,NaN,NaN
98759,199,Something Special,Pop Smoke,NaN,NaN,122,Pop Smoke,NaN,NaN,NaN,...,NaN,NaN,91,Pop Smoke,112,Pop Smoke,NaN,NaN,NaN,NaN


In [29]:
#Now that we have our data compiled and sorted, we can begin looking at it statistically

#total_chart_df.describe() does not really reveal much, but scrolling through the data itself is intriguing.

#So I'm thinking that we have several questions we could examine...

#We could look at some of the top songs in the global playlist and run a test on rank placement
#distribution for countries listed

#in order to do this, we need to create a for loop to get all the rankings of the songs from each country
#in this loop, if the song is in the column, we need to grab the ranks from the rows and append it to a list, ideally we also
#grab the country name as well and put that in a separate list to make it readable/organized

#This is for the first song, ROCKSTAR by DaBaby

#setting empty lists and counter

rockstar_rankings = []

#rockstar_countries = [] #not sure if we'll use this

i=0
for name[i] in total_chart_df_t.iteritems():
    if name == 'ROCKSTAR (feat. Roddy Ricch)':
        rank = name[i-1]
        rockstar_rankings.append(rank)
        i = i+1
print(rockstar_rankings)
#print(rockstar_countries)

#We can look at stream numbers (not listed here but available) relative to population, because that figure
#may be a more accurate representation of popularity potentially


#We could compare based on national tongue (does this have an impact on what songs are popular? - I noticed
#when looking at the data that certain songs were)




TypeError: 'tuple' object does not support item assignment